In [1]:
import pandas as pd
import numpy as np
import os
import requests
import warnings
from unidecode import unidecode

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [2]:
# removes unnamed col
def removeUnnamedCol(df):
    # returns columns that do not start with Unnamed
    return df.loc[:, ~df.columns.str.startswith("Unnamed")]

In [3]:
# takes the player's ids and uses the player dic to create new cols with the player's name
def addPlayerNames(sav, ids_dict):
    sav["batter_name"] = sav["batter"].map(ids_dict)
    sav["pitcher_name"] = sav["pitcher"].map(ids_dict)
    return sav

In [5]:
# create a list of players that are not in the id dictionary
def findNanPlayers(sav):
    missing_pitchers = set(sav.loc[sav["pitcher_name"].isna(), "pitcher"])
    missing_batters = set(sav.loc[sav["batter_name"].isna(), "batter"])
    return list(missing_batters | missing_pitchers)

In [6]:
def getPlayerName_list(pidlist, ids_lookup_df):
    for pid in pidlist:
        url = f"https://statsapi.mlb.com/api/v1/people/{pid}"

        try:
            pagejson = requests.get(url).json()
            playername = pagejson.get("people")[0].get("fullName")
            print(f"Found name {playername} for ID {pid}")
            new_ids_df = pd.DataFrame({"MLBID": [pid], "PLAYERNAME": [playername]})
            ids_lookup_df = pd.concat([ids_lookup_df, new_ids_df], ignore_index=True)
        except Exception as e:
            print(f"ERROR for {pid}: {e}")
            pass
    
    ids_lookup_df = removeUnnamedCol(ids_lookup_df)
    ids_lookup_df.to_csv("data/ids.csv", index=False)

In [7]:
ids_df = pd.read_csv("data/ids.csv", index_col=False)
ids_df = removeUnnamedCol(ids_df)

# create a dictionary of ids and players
id_lookup_dict = dict(zip(ids_df.MLBID, ids_df.PLAYERNAME))

sav25 = pd.read_csv("data/sav25.csv", index_col=False)
sav25 = addPlayerNames(sav25, id_lookup_dict)
nan_list = findNanPlayers(sav25)
getPlayerName_list(nan_list, ids_df)

In [9]:
sav25.to_csv("data/sav25.csv", index=False)

In [10]:
sav25.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches,batter_name,pitcher_name
0,FS,2025-11-01,92.1,-2.02,5.15,"Yamamoto, Yoshinobu",672386,808967,grounded_into_double_play,hit_into_play,NaN,NaN,NaN,NaN,6.0,"Alejandro Kirk grounds into a double play, sho...",W,R,R,TOR,LAD,X,6.0,ground_ball,0,2,2025,-0.48,0.37,0.53,2.17,665489.0,NaN,680718.0,1,11,Bot,119.57,155.13,NaN,NaN,NaN,NaN,7.524095,-133.878517,-2.517287,-7.425629,29.845675,-27.341263,3.13,1.50,5.0,71.9,-24.0,92.2,1320.0,6.5,813024,669257,518692,808975,571970,605141,571771,681909,681624,54.03,0.034,0.029000,0.0,1.0,0.0,0.0,2.0,99,3,Split-Finger,4,5,4,5,5,4,4,5,Standard,Standard,242.0,-0.465,-0.177,73.8,7.7,0.034,0.177,88.0,-1,-1,0.465166,0.465166,26,26,27,27,2,5,1.0,1.0,NaN,NaN,2.33,0.48,0.48,36.9,8.375233,-7.664043,17.979522,48.911447,37.014581,Alejandro Kirk,Yoshinobu Yamamoto
1,CU,2025-11-01,80.3,-1.70,5.45,"Yamamoto, Yoshinobu",672386,808967,NaN,called_strike,NaN,NaN,NaN,NaN,4.0,Called Strike,W,R,R,TOR,LAD,S,NaN,NaN,0,1,2025,1.36,-1.25,-0.28,1.96,665489.0,NaN,680718.0,1,11,Bot,NaN,NaN,NaN,NaN,NaN,NaN,0.561660,-116.768446,1.472301,12.095261,26.350181,-44.003006,2.92,1.37,NaN,NaN,NaN,79.7,2970.0,6.4,813024,669257,518692,808975,571970,605141,571771,681909,681624,54.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,2,Curveball,4,5,4,5,5,4,4,5,Standard,Standard,42.0,-0.018,-0.073,NaN,NaN,NaN,0.073,NaN,-1,-1,0.465166,0.465166,26,26,27,27,2,5,1.0,1.0,NaN,NaN,4.85,-1.36,-1.36,48.1,NaN,NaN,NaN,NaN,NaN,Alejandro Kirk,Yoshinobu Yamamoto
2,FC,2025-11-01,92.8,-1.91,5.24,"Yamamoto, Yoshinobu",672386,808967,NaN,foul,NaN,NaN,NaN,NaN,9.0,Foul,W,R,R,TOR,LAD,S,NaN,NaN,0,0,2025,0.23,0.60,0.56,1.84,665489.0,NaN,680718.0,1,11,Bot,NaN,NaN,NaN,NaN,NaN,NaN,5.856053,-135.072611,-4.233184,1.577981,29.102128,-24.072732,3.13,1.50,1.0,74.8,-53.0,93.2,2446.0,6.5,813024,669257,518692,808975,571970,605141,571771,681909,681624,54.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,1,Cutter,4,5,4,5,5,4,4,5,Standard,Standard,187.0,-0.018,-0.041,73.0,7.5,NaN,0.041,88.0,-1,-1,0.465166,0.465166,26,26,27,27,2,5,1.0,1.0,NaN,NaN,2.04,-0.23,-0.23,39.1,14.162806,5.694338,22.383742,47.652657,23.055298,Alejandro Kirk,Yoshinobu Yamamoto
3,FS,2025-11-01,90.9,-1.97,5.26,"Yamamoto, Yoshinobu",680718,808967,walk,blocked_ball,NaN,NaN,NaN,NaN,14.0,Addison Barger walks.,W,L,R,TOR,LAD,B,NaN,NaN,3,0,2025,-0.69,0.02,0.47,0.83,665489.0,NaN,NaN,1,11,Bot,NaN,NaN,NaN,NaN,NaN,NaN,7.631083,-132.145001,-5.243103,-9.709392,28.153507,-30.982919,3.76,1.79,NaN,NaN,NaN,91.2,1523.0